In [ ]:
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

def CryptoData(symbol, frequency):
    #Params: String symbol, int frequency = 300,900,1800,7200,14400,86400
    #Returns: df from first available date
    url ='https://poloniex.com/public?command=returnChartData&currencyPair='+symbol+'&end=9999999999&period='+str(frequency)+'&start=0'
    df = pd.read_json(url)
    df.set_index('date',inplace=True)
    return df



In [ ]:
# frequency = 86400s which is 1 day
df = CryptoData('USDT_BTC', 86400)['close']
df

In [ ]:
df.plot(figsize = (16,10))

In [ ]:
df.pct_change().describe()

In [ ]:
df.pct_change().hist(bins=100)

In [ ]:
# frequency = 300s which is 5 mins
df = CryptoData(symbol = 'USDT_BTC', frequency = 300)

df['SMA_1000'] = df['close'].rolling(1000).mean()
df['SMA_5000'] = df['close'].rolling(5000).mean()
df[['close','SMA_1000','SMA_5000']][270000:].plot(figsize = (16,10))

In [ ]:
# SMA crossover strategy consists of a leading and a lagging simple moving averages. 
# Leading SMA has a shorter look-back period that lagging moving average. 
# Hence, by definition, leading SMA will be more sensitive to most recent price moves; lagging SMA will be slower to react

# 1. BUY if Leading SMA is above Lagging SMA by some threshold.
# 2. SELL if Leading SMA is below Lagging SMA by some threshold.

# Threshold is applied to filter out weak signals and will be set by default as 2.5% of the current price


import numpy as np

def test_ma(df, lead, lag, pc_thresh = 0.025):
    ma_df = df.copy()
    ma_df['lead'] = ma_df['close'].rolling(lead).mean()
    ma_df['lag'] = ma_df['close'].rolling(lag).mean()
    ma_df.dropna(inplace = True)
    ma_df['lead-lag'] = ma_df['lead'] - ma_df['lag']
    ma_df['pc_diff'] = ma_df['lead-lag'] / ma_df['close']

    # regime column will govern the buying and selling logic as described above; 
    # 1 means that we are long, -1 means that we are short and 0 means no position
    ma_df['regime'] = np.where(ma_df['pc_diff'] > pc_thresh, 1, 0)
    ma_df['regime'] = np.where(ma_df['pc_diff'] < -pc_thresh, -1, ma_df['regime'])
    
    # Market column as log returns of price series - which is the market performance
    ma_df['Market'] = np.log(ma_df['close'] / ma_df['close'].shift(1))
    
    # compute our Strategy returns by multiplying regime (shifted forward to match the Market column
    ma_df['Strategy'] = ma_df['regime'].shift(1) * ma_df['Market']
    
    # perform a cumulative sum operation as well as apply an exponent on Market and Strategy log returns 
    # in order to recover the original normalised series
    ma_df[['Market','Strategy']] = ma_df[['Market','Strategy']].cumsum().apply(np.exp)
    return ma_df

In [ ]:
ma_df = test_ma(df, 1000, 5000).dropna()

ma_df['regime'].plot(figsize=(16,5))

In [ ]:
ma_df[['Strategy','Market']].iloc[-1]

In [ ]:
ma_df[['Strategy','Market']][200000:].plot(figsize = (16,10))